# 분석 목표
고객의 중분류 별로 선호하는 ETF를 확인하고, 그 ETF들의 특성을 나누어서 각 군집별로 추천하려고 합니다.

In [ ]:
# 필요 library 설치
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os #데이터 불러올 시 데이터 경로를 맞추기 위함

In [ ]:
# 데이터 불러오기
## 파일 명을 알기 쉽게 정리
해외종목정보 = pd.read_csv("NH_CONTEST_NW_FC_STK_IEM_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
종목일자별시세 = pd.read_csv("NH_CONTEST_STK_DT_QUT.csv", encoding="EUC-KR", skipinitialspace=True)
주식일별정보 = pd.read_csv("NH_CONTEST_NHDATA_STK_DD_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
매수매도계좌정보 = pd.read_csv("NH_CONTEST_NHDATA_IFW_OFW_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
고객보유정보 = pd.read_csv("NH_CONTEST_NHDATA_CUS_TP_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
ETF구성종목정보 = pd.read_csv("NH_CONTEST_DATA_ETF_HOLDINGS.csv", encoding="EUC-KR", skipinitialspace=True)
ETF배당내역 = pd.read_csv("NH_CONTEST_DATA_HISTORICAL_DIVIDEND.csv", encoding="EUC-KR", skipinitialspace=True)
ETF점수정보 = pd.read_csv("NH_CONTEST_ETF_SOR_IFO.csv", encoding="EUC-KR", skipinitialspace=True)

data_name = ["해외종목정보", "종목일자별시세", "주식일별정보", "매수매도계좌정보", "고객보유정보", "ETF구성종목정보", "ETF배당내역", "ETF점수정보"]

## 각 파일 내 변수명 변경 + 공백제거
해외종목정보.columns = ["티커종목코드", "외화증권한글명", "외화증권영문명", "주식/ETF구분코드",
                   "상장주식총수량", "외화시장구분코드", "회사주소", "웹주소", "업종분류명",
                   "CEO명", "영문사업개요내용", "섹터분류명", "산업명", "시가총액"]
해외종목정보 = 해외종목정보.map(lambda x: x.strip() if isinstance(x, str) else x)

종목일자별시세.columns = ["거래일자", "티커종목코드", "종목시가", "종목고가", "종목저가",
                   "종목종가", "전일대비증감가격", "전일대비증감율", "누적거래수량",
                   "거래대금", "매도체결합계수량", "매수체결합계수량", "환율"]
종목일자별시세 = 종목일자별시세.map(lambda x: x.strip() if isinstance(x, str) else x)

주식일별정보.columns = ["거래일자", "티커종목코드", "총보유계좌수", "총보유수량",
                   "당사평균보유수량", "당사평균보유비중비율", "당사평균평가손익",
                   "당사평균매입단가", "당사평균수익율", "당사평균보유기간일수",
                   "분포상위10퍼센트수치", "분포상위30퍼센트수치", "분포상위50퍼센트수치",
                   "분포상위70퍼센트수치", "분포상위90퍼센트수치", "기준종가", "손실투자자비율",
                   "수익투자자비율", "신규매수계좌수", "전량매도계좌수", "종목조회건수",
                   "관심종목등록건수"]
주식일별정보 = 주식일별정보.map(lambda x: x.strip() if isinstance(x, str) else x)

매수매도계좌정보.columns = ["기준일자", "티커종목코드", "유입/유출구분코드", "유입유출티커코드",
                    "유입유출금액비중", "유입유출랭크"]
매수매도계좌정보 = 매수매도계좌정보.map(lambda x: x.strip() if isinstance(x, str) else x)

고객보유정보.columns = ["기준일자", "티커종목코드", "고객구성대분류코드", "고객구성중분류코드",
                  "고객구성계좌수비율", "고객구성투자비율"]
고객보유정보 = 고객보유정보.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF구성종목정보.columns = ["대상ETF티커", "ETF개별구성종목티커", "보유종목의가치(USD)", "보유종목의영문명", "보유종목의한글명",
                     "보유종목의주수(주)", "보유종목의비중(%)", "보유종목의타입(ST:주식,EF:ETF,EN:ETN,SSEF:Single-StockETF)"]
ETF구성종목정보 = ETF구성종목정보.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF배당내역.columns = ["대상ETF티커", "배당락일(YYYYMMDD)", "배당금", "수정배당금", "배당기준일(YYYYMMDD)",
                   "지급일(YYYYMMDD)", "공시일(YYYYMMDD)", "배당주기(Quarterly:분기배당,Weekly:주배당,Monthly:월배당,SemiAnnual:반기배당,Annual:연배당,Other:알수없음)"]
ETF배당내역 = ETF배당내역.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF점수정보.columns = ["거래일자", "대상ETF티커", "1개월총수익율", "3개월총수익율",
                   "1년총수익율", "ETF점수", "ETFZ점수", "Z점수순위",
                   "누적수익율Z점수", "정보비율Z점수", "샤프지수Z점수", "상관관계Z점수",
                   "트래킹에러Z점수", "최대낙폭Z점수", "변동성Z점수"]
ETF점수정보 = ETF점수정보.map(lambda x: x.strip() if isinstance(x, str) else x)

## 설명
### 코드의 목표
거래일자와 고객 중분류를 검색 key로 삼아 해당 거래일자와 해당 고객 중분류가 각 ETF를 얼마나 보유할 수 있는지 검색할 수 있는 dictionary 제작하였습니다.

### 코드 작동 설명
고객보유정보에서 각 거래일자에서 ETF를 특정 고객 집단(대분류)이 보유하고 있는 비율을 확인할 수 있습니다.
주식일별정보에서는 각 거래일자에서 ETF를 보유하고 있는 계좌가 총 몇 개인지, 그리고 고객이 보유하고 있는 총 ETF의 수가 몇 개 인지 확인할 수 있습니다.
비율과 총 수량을 곱해 특정 거래일자에 한 집단에서 몇 개의 계좌가 ETF를 보유하고 있는지, 총 보유하고 있는 ETF는 몇 개 인지 확인할 수 있는 딕셔너리를 제작하였습니다.

In [ ]:
# 원하는 날짜의 거래를 필터링 하는 함수
def filter_date(df, date):
    if '기준일자' in df.columns:
        return df[df['기준일자'] == date]
    elif '거래일자' in df.columns:
        return df[df['거래일자'] == date]

# 원하는 종목을 필터링 하는 함수
def filter_code(df, ticker_code):
    if '티커종목코드' in df.columns:
        return df[df['티커종목코드'] == ticker_code]
    elif '대상ETF티커' in df.columns:
        return df[df['대상ETF티커'] == ticker_code]

# 고객 구성 대분류를 필터링 하는 함수
def filter_bigConstraint(df, big_constraint):
    return df[df['고객구성대분류코드'] == big_constraint]

# 날짜, 고객 별로 ETF를 얼마나 갖고 있는지 저장할 dictionary
# (거래일자, 고객중분류) 를 key로 받아 해당 날짜에 해당 고객 중분류가 각 ETF를 얼만큼 보유하고 있는지 확인 가능.
dictionary_by_customer_date = {}

def add_or_concat(key, new_data):
    # 데이터프레임 생성 (new_data는 새로운 row을 담은 DataFrame)
    new_df = pd.DataFrame(new_data)

    # 해당 key에 데이터프레임이 없는 경우, 새로운 데이터프레임 추가
    if key not in dictionary_by_customer_date:
        dictionary_by_customer_date[key] = new_df
    else:
        # 이미 데이터프레임이 있다면, pd.concat으로 row 추가
        dictionary_by_customer_date[key] = pd.concat([dictionary_by_customer_date[key], new_df])

# 특정 거래일자에, 특정 고객 중분류가 각 ETF를 얼만큼 갖고 있는지 확인하는 함수
def calculate_actual_numbers(고객보유정보, 주식일별정보, date, ticker_code, big_constraint):
    filtered_stock_info = filter_date((filter_code(주식일별정보, ticker_code)), date)
    rowIndex_stock_info = filtered_stock_info.index[0]
    filtered_customer_info = filter_bigConstraint(filter_date((filter_code(고객보유정보, ticker_code)), date), big_constraint)
    for rowIndex in filtered_customer_info.index:
        accounts = filtered_customer_info.loc[rowIndex, "고객구성계좌수비율"] * filtered_stock_info.loc[rowIndex_stock_info, "총보유계좌수"]
        stocks = filtered_customer_info.loc[rowIndex, "고객구성투자비율"] * filtered_stock_info.loc[rowIndex_stock_info, "총보유수량"]
        dont_know_proper_name = pd.DataFrame({"보유고객계좌수":[accounts], "보유고객주식수":[stocks]}, index=[ticker_code])
        add_or_concat((date, filtered_customer_info.loc[rowIndex, "고객구성중분류코드"]), dont_know_proper_name)

# 주식 일별 정보를 제공하는 ETF만 확인
# 아래 데이터를 가공하는 코드는 3시간 40분 걸림.
tickerCode = list(주식일별정보['티커종목코드'].unique())

for tC in tickerCode:
    imsi = 주식일별정보[주식일별정보['티커종목코드'] == tC]
    ETF_dates = list(imsi['거래일자'].unique())
    for date in ETF_dates:
        for i in range(1,4): #고객 대분류 번호가 1, 2, 3
            calculate_actual_numbers(고객보유정보, 주식일별정보, date, tC, i)
            print(f"{date}, {tC}, 대분류 :{i}")

# 파일을 만들어서 딕셔너리 저장
file_path = f"날짜및고객분류별보유ETF.pkl"
with open(file_path, 'wb') as f:
    pickle.dump(dictionary_by_customer_date, f)

In [ ]:
# 이미 만들어 놓은 딕셔너리를 불러와서 사용할 수 있도록 함.
with open('날짜및고객분류별보유ETF.pkl', 'rb') as f:
    dictionary_by_customer_date = pickle.load(f)

# 딕셔너리 예시 확인
dictionary_by_customer_date[(20240726, 11)]
#2024년 7월 26일에, 11(투자고수)의 ETF 보유 목록 확인.

# ETF 테이블 제작
여기는 뭐라고 설명하지?

ETF 테이블을 만드는 과정에서, EDA를 같이 진행하였습니다.

## ETF EDA

In [ ]:
# ETF점수정보 파일에 들어있는 column 명을 확인해보았습니다.
ETF점수정보.columns

In [ ]:
# 해외종목정보 파일에, ETF 외에 주식도 포함 되어 있는 것을 확인하여,
# ETF만을 필터링하였습니다.
ETF해외종목정보 = 해외종목정보[해외종목정보["주식/ETF구분코드"] == "ETF"]

In [ ]:
# 주식일별 정보 중, 앞서 필터링한 ETF해외종목정보가 다 들어 있는지 확인
print(주식일별정보["티커종목코드"].unique().shape)
print(ETF해외종목정보["티커종목코드"].unique().shape)
np.isin(주식일별정보["티커종목코드"].unique(), ETF해외종목정보["티커종목코드"].unique()).sum()

# ETF해외종목정보에는 167개의 종목이 있지만, 그 중 주식일별정보에 있는 ETF는 152개
# 일부 종목은 주식일별정보에 존재하지 않는 것을 확인할 수 있었습니다.

In [ ]:
print(종목일자별시세["티커종목코드"].unique().shape)
print(ETF해외종목정보["티커종목코드"].unique().shape)
np.isin(종목일자별시세["티커종목코드"].unique(), ETF해외종목정보["티커종목코드"].unique()).sum()

# 비슷하게 종목일자별시세에도 확인,
# 모든 종목이 종목일자별시세에는 존재하는 것을 확인할 수 있었습니다.

In [ ]:
print(ETF해외종목정보["티커종목코드"].unique().shape)
print(ETF구성종목정보["대상ETF티커"].unique().shape)
np.isin(ETF해외종목정보["티커종목코드"].unique(), ETF구성종목정보["대상ETF티커"].unique()).sum()

# ETF구성종목정보에 있는 모든 ETF 종목이,
# ETF해외종목정보에 전부 들어있는 것을 확인하였습니다.

In [ ]:
print(ETF배당내역["대상ETF티커"].unique().shape)
print(ETF구성종목정보["대상ETF티커"].unique().shape)
np.isin(ETF배당내역["대상ETF티커"].unique(), ETF구성종목정보["대상ETF티커"].unique()).sum()

# 그러나 ETF구성종목정보에 있는 모든 종목이, ETF배당내역에 있지는 않음을 확인하였습니다.

In [ ]:
print(ETF배당내역["대상ETF티커"].unique().shape)
print(ETF점수정보["대상ETF티커"].unique().shape)
np.isin(ETF배당내역["대상ETF티커"].unique(), ETF점수정보["대상ETF티커"].unique()).sum()

# 점수 정보를 제공하는 ETF가, ETF배당내역에 있는지 확인하였습니다.
# 한 종목이 비어있는 것을 확인하였습니다.

In [ ]:
bool_array = np.isin(ETF점수정보["대상ETF티커"].unique(), ETF배당내역["대상ETF티커"].unique())
ETF점수정보["대상ETF티커"].unique()[np.where(~bool_array)[0]]

# 위에서 비어있는 종목을 확인해보니,
# 배당날짜가 21년이라 없는 것을 확인하였습니다.

In [ ]:
print(ETF구성종목정보["대상ETF티커"].unique().shape)
print(ETF점수정보["대상ETF티커"].unique().shape)
np.isin(ETF구성종목정보["대상ETF티커"].unique(), ETF점수정보["대상ETF티커"].unique()).sum()

# ETF점수정보를 제공해주는 ETF가, ETF구성종목정보에 모두 들어있지는 않음을 확인하였습니다.

## ETF 테이블 생성
본격적으로 분석에 사용할 ETF 테이블을 만들어 보았습니다.

In [ ]:
# NH에서 제공해주는 데이터에 맞추기 위해 주식일별정보를 베이스로 사용하였습니다.  
etf_df = 주식일별정보.copy()
etf_df = etf_df[np.isin(etf_df["티커종목코드"], ETF해외종목정보["티커종목코드"])]
etf_df['티커종목코드'].unique().shape
# 만들어질 ETF 테이블에 들어갈 ETF 종목 수를 확인하였습니다.

In [ ]:
# 주식일별정보와 ETF해외종목정보를 merge 하였습니다.
etf_df = pd.merge(etf_df, ETF해외종목정보[["티커종목코드", "외화증권영문명"]], on='티커종목코드', how='left')
first_cols = ["티커종목코드", "외화증권영문명", "거래일자"]
other_cols = [col for col in etf_df.columns if col not in first_cols]
etf_df = etf_df[first_cols + other_cols]
# etf_df = etf_df[["티커종목코드", "외화증권영문명", "거래일자", "당사평균매입단가", "당사평균보유기간일수", "관심종목등록건수", "종목조회건수"]]
print(etf_df.info())
etf_df = etf_df.sort_values(by=["티커종목코드", '거래일자'])

In [ ]:
# 앞서 만든 etf_df 데이터 프레임에 종목일자별시세를 merge 하였습니다.
etf_df = pd.merge(etf_df, 종목일자별시세[["거래일자", "티커종목코드", "종목종가", "전일대비증감율", "누적거래수량", "거래대금", "매도체결합계수량", "매수체결합계수량"]], on=['티커종목코드', '거래일자'], how='left')
print(etf_df.info()) # null 값이 470개가 있음을 확인하였습니다.

In [ ]:
# 앞서 만든 etf_df 데이터 프레임에 ETF 점수정보를 merge 하였습니다.
etf_df = pd.merge(etf_df, ETF점수정보, left_on=['티커종목코드', '거래일자'], right_on=['대상ETF티커', '거래일자'], how='left')
print(etf_df.info())

In [ ]:
# 이중 index를 활용하여 데이터프레임을 완성하였습니다.
df = etf_df.set_index(['티커종목코드', '거래일자'])
# 만든 데이터 프레임을 저장하였습니다.
df.to_csv('etf_df.csv', encoding='utf-8-sig') 
df

In [ ]:
# 아래와 같이 종목과 특정 거래일자의 거래 정보를 검색할 수 있습니다.
df.loc[('SPY', 20240528)]